Input: 
> 沖壓機清洗後資料

> 瞬測儀清洗後資料

Output: 
> X_train: training data (data_num*rate, sequence_num, feature_num)

> Y_train: training label (data_num*rate, output_num)

> X_test: testing data (data_num*(1-rate), sequence_num, feature_num)

> Y_test: testing label (data_num*(1-rate), output_num)

In [26]:
import pandas as pd
import numpy as np
from random import sample
import random
from datetime import datetime

# below is for loading data from google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sequence_num = 100   #資料期數
time_num = 3600      #沖壓機與瞬測儀的時間差距(second)
print("Sequence number: ", sequence_num)
print("Time number: ", time_num)

# put your folder path, ex: '/content/drive/MyDrive/.../'
folder_path = 'folderpath'

# put your data path, ex: '/content/drive/MyDrive/.../'
input_data_machine = '沖壓機 file path' 
input_data_rate = '瞬測儀 file path'

data_machine = pd.read_excel(input_data_machine, usecols=["編號","frequency","Speed","Status"])
data_rate = pd.read_excel(input_data_rate)
data_machine = data_machine.dropna()
data_rate = data_rate.dropna()

print(data_machine.head())
print(data_rate.head())
print(data_machine.shape)
print(data_rate.shape)

Mounted at /content/drive
Sequence number:  100
Time number:  3600
                              編號  Speed  Status  frequency
0  2020081008:11_D-008_101180441  361.0     2.0   3.000000
1  2020081008:12_D-008_101180441  361.0     2.0   3.500000
2  2020081008:13_D-008_101180441  361.0     2.0   3.516667
3  2020081008:14_D-008_101180441  250.0     4.0   1.800000
4  2020081008:19_D-008_101180441  250.0     2.0   0.050000
                       產生檢驗單號的時間            料號  最大 - detail_5  最大 - detail_6  \
0  2020082114:46_D-006_101181727  0162B00200-1         2.1900           2.20   
1  2020082121:39_D-006_101181727  0162B00200-1         2.1900           2.21   
2  2020082205:10_D-006_101181727  0162B00200-1         2.1900           2.20   
3  2020082208:44_D-006_101181727  0162B00200-1         2.2000           2.20   
4  2020082421:10_D-008_101181721    0162B00200         3.2853           2.25   

   最大 - detail_7  最大 - detail_8  最大 - detail_9  最大 - detail_10  \
0         2.2000         2.2000 

In [18]:
# 只抓特定料號
data_rate = data_rate[data_rate["料號"] == "0162B00100"]

In [19]:
"""
將瞬測儀和機台數據資料分包&配對
"""
data_machine.set_index("編號", inplace=True)
all_data_machineId = np.array(data_machine.index.drop_duplicates(keep='first').values)
data_machine.reset_index(inplace=True)

data_rate.set_index("產生檢驗單號的時間", inplace=True)
all_data_rateId = np.array(data_rate.index.drop_duplicates(keep='first').values)
data_rate.reset_index(inplace=True)

print("沖壓機不重複編號個數: ",all_data_machineId.size)
print("瞬測儀不重複編號個數: ",all_data_rateId.size)

pkg_num = 0

for data_rateId in all_data_rateId:
  time_compare = time_num
  data_machineId_compare = 0
  for data_machineId in all_data_machineId:
    """
    完全相同
    """
    # if data_machineId == data_rateId:
    #   pkg_num += 1 
    #   globals()['x_'+str(pkg_num)] = data_machine[data_machine["編號"] == data_machineId]
    #   globals()['y_'+str(pkg_num)] = data_rate[data_rate["產生檢驗單號的時間"] == data_rateId]
    # else:
    """
    完全相同&時間誤差time_num內視為同包
    """
    machineId = data_machineId[14:29]   #抓機台編號_工單號
    rateId = data_rateId[14:29]         #抓機台編號_工單號
    
    if rateId == machineId:
      time_machine = datetime.strptime(data_machineId[0:13], "%Y%m%d%H:%M")
      time_rate = datetime.strptime(data_rateId[0:13], "%Y%m%d%H:%M")
      time_diff = (time_rate - time_machine).total_seconds()  #計算時間誤差

      if time_diff <= time_num and time_diff >= 0 and time_diff < time_compare:
        data_machineId_compare = data_machineId
        time_compare = time_diff
          
  if data_machineId_compare != 0:
    pkg_num += 1 
    globals()['x_'+str(pkg_num)] = data_machine[data_machine["編號"] == data_machineId_compare]    #沖壓機資料
    globals()['y_'+str(pkg_num)] = data_rate[data_rate["產生檢驗單號的時間"] == data_rateId]         #瞬測儀資料

max_pkg_num = pkg_num
print("Total package number: ", max_pkg_num)

沖壓機不重複編號個數:  351091
瞬測儀不重複編號個數:  566
Total package number:  489


In [20]:
"""
機台資料往前取同機台工單sequence_num筆，計算筆數小於sequence_num筆包數
"""
usable_pkg = 0
lessthan = 0

for pkg_num in range(1, max_pkg_num+1):
  rateId = str(globals()['y_'+str(pkg_num)][["產生檢驗單號的時間"]].values)[17:32] #抓機台編號_工單號
  machineIndex = int(globals()['x_'+str(pkg_num)].index[0])                     #抓對應的沖壓機index
  machineId = str(globals()['x_'+str(pkg_num)][["編號"]].values)[17:32]          #抓機台編號_工單號

  for num in range(machineIndex,0,-1):
    if num == machineIndex:
      globals()['x_sequence_'+str(pkg_num)] = data_machine.loc[[num]]
    else: #往前取同機台工單號sequence_num筆
      if (str(data_machine["編號"][num])[14:31] == machineId) and (len(globals()['x_sequence_'+str(pkg_num)]) < sequence_num):
        globals()['x_sequence_'+str(pkg_num)] = pd.concat([globals()['x_sequence_'+str(pkg_num)], data_machine.loc[[num]]])

for pkg_num in range(1, max_pkg_num+1):
  if len(globals()['x_sequence_'+str(pkg_num)]) < sequence_num:
    lessthan += 1
  else:
    if usable_pkg == 0:
      first_pkg = pkg_num 
    usable_pkg += 1
  globals()['x_sequence_'+str(pkg_num)] = globals()['x_sequence_'+str(pkg_num)].drop(["編號"], axis=1)
  globals()['y_sequence_'+str(pkg_num)] = globals()['y_'+str(pkg_num)].drop(["產生檢驗單號的時間","料號"], axis=1)

print("Less than sequence_num data package: ",lessthan)
print("Usable package: ",usable_pkg)
print("First package number: ",first_pkg)
print(globals()['x_sequence_'+str(first_pkg)].shape)  #(sequence_num, feature_num)
print(globals()['y_sequence_'+str(first_pkg)].shape)  #(1, output_num)

Less than sequence_num data package:  11
Usable package:  478
First package number:  1
(100, 3)
(1, 26)


In [21]:
"""
依序存進data & label, 將pd格式轉為np格式
"""
for pkg_num in range(1, max_pkg_num+1):
  if len(globals()['x_sequence_'+str(pkg_num)]) < sequence_num:
    pass
  else:
    if pkg_num == first_pkg:
      data = pd.DataFrame(globals()['x_sequence_'+str(pkg_num)])   
      label = pd.DataFrame(globals()['y_sequence_'+str(pkg_num)])    
    else:
      data = pd.concat([data, pd.DataFrame(globals()['x_sequence_'+str(pkg_num)])])
      label = pd.concat([label, pd.DataFrame(globals()['y_sequence_'+str(pkg_num)])])

data = data.values
label = label.values

print(data.shape)     #(data_num*sequence_num, feature_num)
print(label.shape)    #(data_num, output_num)

(47800, 3)
(478, 26)


In [22]:
"""
將data從(data_num*sequence_num, feature_num)轉為(data_num, sequence_num, feature_num)
"""
slide_size = 0
pkg_size = 0
data_temp = []

for pkg_num in range(1, max_pkg_num+1):
  if len(globals()['x_sequence_'+str(pkg_num)]) < sequence_num:
    pass
  else: 
    data_temp.append(data[slide_size:(slide_size+sequence_num), 0:3])
    slide_size += sequence_num

data = np.array(data_temp)

print(data.shape)     #(data_num, sequence_num, feature_num)
print(label.shape)    #(data_num, output_num)

(478, 100, 3)
(478, 26)


In [23]:
"""
將所有資料shuffle，並分為train & test data
"""
permutation = np.random.permutation(label.shape[0])
shuffled_data = data[permutation,:,:]
shuffled_label = label[permutation]

rate=0.7 #train & test data區分比例
X_train = shuffled_data[:int(shuffled_data.shape[0]*rate)]
Y_train = shuffled_label[:int(shuffled_label.shape[0]*rate)]
X_test = shuffled_data[int(shuffled_data.shape[0]*rate):]
Y_test = shuffled_label[int(shuffled_label.shape[0]*rate):]

print(X_train.shape)  #(data_num*rate, sequence_num, feature_num)
print(Y_train.shape)  #(data_num*rate, output_num)
print(X_test.shape)   #(data_num*(1-rate), sequence_num, feature_num)
print(Y_test.shape)   #(data_num*(1-rate), output_num)

(334, 100, 3)
(334, 26)
(144, 100, 3)
(144, 26)


In [27]:
"""
將train & test data儲存為npy檔
"""
np.save(folder_path + 'x_train.npy', X_train)
np.save(folder_path + 'y_train.npy', Y_train)
np.save(folder_path + 'x_test.npy', X_test)
np.save(folder_path + 'y_test.npy', Y_test)
print("Done")

Done
